In [1]:
import random

In [2]:
def choose(n, k):
    """
    A fast way to calculate binomial coefficients by Andrew Dalke (contrib).
    """
    if 0 <= k <= n:
        ntok = 1
        ktok = 1
        for t in range(1, min(k, n - k) + 1):
            ntok *= n
            ktok *= t
            n -= 1
        return ntok // ktok
    else:
        return 0

In [3]:
class Groups:
    def __init__(self, players_list):
        self.players = set(players_list)
        self.groups = self.random_grps(self.players)
        self.checkpoint = []
        #self.max_groups = self.max_groups(len(self.players))
        if not self.groups:
            raise ValueError("random groups not initiated")

    def __str__(self):
        return "Groups: {}".format(self.groups)
    
    def loss(self):
        return 0 # replace with function to minimize

    def random_grps(self, players_list, grouped=None):
        """
        Takes a set of players, returns a list of groups {set}
        """
        players = players_list.copy()
        if len(players) == 1:
            return []
        if len(players) == 3:
            try:
                grouped.append(players)
            except AttributeError:
                grouped = [players]
            return grouped
        if not players:
            return grouped

        max_players = 10 if len(players) > 9 else len(players) + 1
        grp_size = random.randrange(2, max_players)
        # choose random numbers until grp_size valid
        while (len(players) - grp_size == 1):
            grp_size = random.randrange(2, max_players)
            
        group = set(random.sample(players, grp_size))
        if not grouped:
            grouped = [group]
        else:
            grouped.append(group)
        return self.random_grps(players.difference(group), grouped)

    # Acts on data members
    def random_swap(self, group1, group2):
        player1 = random.choice(list(group1))
        player2 = random.choice(list(group2))
        g1, g2 = group1.copy(), group2.copy()
        _replace(group1, player1, player2)
        _replace(group2, player2, player1)
        assert(g1!=group1)
        assert(g2!=group2)
        self.checkpoint = [(g1, group1), (g2, group2)]
    
    def undo_swap(self):
        old_g1, curr_g1 = self.checkpoint[0]
        old_g2, curr_g2 = self.checkpoint[1]
        _replace(self.groups, curr_g1, old_g1)
        _replace(self.groups, curr_g2, old_g2)
        assert(old_g1 in self.groups)
        assert(old_g2 in self.groups)

    def max_groups(self, num_players):
        if num_players == 2 or num_players == 3:
            return 1
        maxn = 0
        for i in range(num_players//2 + 1):
            if i == (num_players-1) or i == 1:
                continue
            if i == 0 and num_players <= 9:
                maxn += 1
            else:
                # eliminate complementary chooses, so halve 4C2, 6C3, etc.
                comb = (choose(num_players, i) if (i != num_players / 2)
                        else choose(num_players, i) // 2)
                maxn += comb * self.max_groups(num_players-i)
        return maxn
    
def _replace(container, elem1, elem2):
    container.remove(elem1)
    try:
        container.add(elem2)
    except AttributeError:
        container.append(elem2)

In [4]:
class SimulatedAnnealing:
    def __init__(self, players_list, has_criteria=False):
        """ If we also care about other criteria besides uniqueness, has_criteria=True"""
        self.start_state = Groups(players_list)
        self.previous_groups = [self.start_state]
        self.current_state = Groups(players_list)
        self.other_criteria = has_criteria
    
    def _find_non_unique(self, state):
        return [group for group in state.groups if group in self.previous_groups]
    
    def generate(self):
#         if not self.other_criteria:
        yield self.start_state
        while True:
            non_unique = self._find_non_unique(self.current_state)
            while non_unique:
                if len(non_unique) >= 2:
                    group1, group2 = random.sample(non_unique, 2)
                else:
                    group1 = non_unique[0]
                    group2 = random.choice(self.current_state.groups)
                self.current_state.random_swap(group1, group2)
                non_unique = self._find_non_unique(self.current_state)
            self.previous_groups.append(self.current_state)
            yield self.current_state
    
    def energy(self, state):
        for group in state.groups:
            if group in self.previous_groups:
                return float('inf')
        return 0

In [5]:
players = {1, 3, 5, 6, 9, 7, 11, 12, 8, 4}

In [6]:
g = Groups(players)

In [7]:
print(g)

Groups: [{3, 4, 6, 7, 8, 11, 12}, {1, 5, 9}]


In [8]:
g.random_swap(g.groups[0], g.groups[1])
print(g)

Groups: [{3, 4, 5, 6, 7, 11, 12}, {8, 1, 9}]


In [9]:
g.undo_swap()
print(g)

Groups: [{3, 4, 6, 7, 8, 11, 12}, {1, 5, 9}]


In [10]:
s = SimulatedAnnealing(players)

In [11]:
print(s.start_state)

Groups: [{1, 5, 7}, {11, 4, 12}, {8, 6}, {9, 3}]


In [12]:
y = s.generate()

In [13]:
print(next(y))

Groups: [{1, 5, 7}, {11, 4, 12}, {8, 6}, {9, 3}]
